In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [11]:
bond1 = 'Price History_20+_US_TBills.xlsx'
bond2 = 'Price History_7-10_US_TBills.xlsx'
equity = 'Price History_IVV_US_Equity.xlsx'
gold = 'Price History_IAU_Gold_Trust.xlsx'
# Load the Excel file into a DataFrame
df = pd.read_excel(bond1)


In [12]:
def load_data(file_paths):
    """
    Load and combine data from multiple .xlsx files.
    Args:
        file_paths (list): List of file paths to .xlsx files.
    Returns:
        pd.DataFrame: Combined DataFrame with all asset data.
    """
    all_data = []
    for file_path in file_paths:
        df = pd.read_excel(file_path)  # Load .xlsx file
        asset_name = file_path.split('/')[-1].split('.')[0]  # Extract asset name from filename
        df['Asset'] = asset_name  # Add asset name column
        all_data.append(df)
    
    # Combine all files into a single DataFrame
    combined_data = pd.concat(all_data, ignore_index=True)
    return combined_data

In [27]:
def dynamic_rebalancing_portfolio(file_paths, initial_money, rebalance_period='M', top_assets=2):
    """
    Implements dynamic rebalancing of a portfolio.
    Args:
        file_paths (list): List of file paths to the Excel files for each asset.
        initial_money (float): Total initial amount of money to invest.
        rebalance_period (str): Period for rebalancing ('M' for monthly, 'W' for weekly, etc.).
        top_assets (int): The number of assets to select for rebalancing.
    Returns:
        dict: Final portfolio value, asset weights, and performance metrics.
    """
    # Load the data from Excel files
    all_data = []
    for file_path in file_paths:
        data = pd.read_excel(file_path)
        data['Exchange Date'] = pd.to_datetime(data['Exchange Date'], errors='coerce')
        data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
        data = data.dropna(subset=['Exchange Date', 'Close'])
        data = data.sort_values(by='Exchange Date')  # Sort by date
        data = data.loc[~data['Exchange Date'].duplicated(keep='first')]  # Remove duplicate dates
        all_data.append(data[['Exchange Date', 'Close']])

    # Merge all data on Exchange Date
    data = pd.concat(all_data, axis=1, keys=[f"Asset_{i}" for i in range(len(all_data))])
    data.columns = pd.MultiIndex.from_tuples(
        [(col[0], col[1]) for col in data.columns],
        names=["Asset", "Field"]
    )
    data = data.dropna()  # Remove rows with missing values

    # Resample the data to the desired rebalance period
    data_resampled = data.resample(rebalance_period).last()

    # Initialize variables
    portfolio_value = initial_money
    portfolio_weights = {}
    
    # Perform dynamic rebalancing
    for current_date in data_resampled.index:
        # Get data up to the current rebalance date
        current_data = data.loc[:current_date]
        
        # Calculate daily returns for each asset up to the rebalance date
        asset_returns = current_data.groupby('Asset')['Close'].pct_change().iloc[-1]
        
        # Select the top assets based on returns
        top_assets_selected = asset_returns.nlargest(top_assets).index
        
        # Allocate portfolio equally among selected assets
        num_selected_assets = len(top_assets_selected)
        weight = 1 / num_selected_assets
        for asset in top_assets_selected:
            portfolio_weights[asset] = weight
        
        # Update portfolio value (assuming equal allocation in the top assets)
        portfolio_value = portfolio_value * (1 + asset_returns.loc[top_assets_selected].sum())

    # Metrics and results
    starting_value = initial_money
    ending_value = portfolio_value
    total_days = (data_resampled.index[-1] - data_resampled.index[0]).days
    total_years = total_days / 365.0

    # 1. CAGR
    cagr = ((ending_value / starting_value) ** (1 / total_years)) - 1
    
    # 2. Volatility (Standard Deviation of Daily Returns)
    volatility = asset_returns.std() * np.sqrt(252)  # Annualized volatility
    
    # 3. Max Drawdown
    rolling_max = data_resampled.cummax()
    drawdowns = (data_resampled - rolling_max) / rolling_max
    max_drawdown = drawdowns.min()
    
    # 4. Sharpe Ratio (assuming risk-free rate = 0 for simplicity)
    sharpe_ratio_value = asset_returns.mean() / asset_returns.std() * np.sqrt(252)
    
    # 5. Value at Risk (99% Confidence)
    var_99 = asset_returns.quantile(0.01)  # 1st percentile of returns

    # Plot portfolio value over time
    plt.figure(figsize=(14, 7))
    plt.plot(data_resampled.index, data_resampled['Portfolio Value'], label='Portfolio Value (Dynamic Rebalancing)', color='green')
    plt.title("Portfolio Growth Over Time (Dynamic Rebalancing)")
    plt.xlabel("Date")
    plt.ylabel("Portfolio Value (USD)")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

    # Return metrics and results
    return {
        "Final Portfolio Value": f"${ending_value:,.2f}",
        "Portfolio Weights": portfolio_weights,
        "CAGR": f"{cagr:.2%}",
        "Volatility": f"{volatility:.2%}",
        "Max Drawdown": f"{max_drawdown:.2%}",
        "Sharpe Ratio": f"{sharpe_ratio_value:.2f}",
        "99% VaR": f"{var_99:.2%} (daily loss threshold)"
    }

In [28]:
capital = 1000000
files = [gold, equity]

In [29]:
dynamic_rebalancing_portfolio(files, capital, rebalance_period='M', top_assets=2)


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'